**Compare extractive_summary and  abstractive_summary**

In [2]:
SELECT
    id,
    azure_cognitive.summarize_extractive(description, 'en', 2) AS extractive_summary,
    azure_cognitive.summarize_abstractive(description, 'en', 2) AS abstractive_summary
FROM listings
WHERE id IN (1, 2);

(2 row(s) affected)

Total execution time: 00:00:17.062

id,extractive_summary,abstractive_summary
1,"{""(\""Make your self at home in this charming one-bedroom apartment, centrally-located on the west side of Queen Anne hill.\"",1)"",""(\""The apartment opens onto a private deck, complete with it's own BBQ, overlooking a garden and a forest of black bamboo.\"",0.68)""}","[""This charming one-bedroom apartment, located on the west side of Queen Anne hill, offers a blend of elegance and convenience in a centrally-located, private space within a duplex. The unit features an open floor plan, bamboo flooring, a fully-equipped kitchen, and a cozy bedroom with a queen-sized bed, capable of accommodating up to four individuals. Additional amenities include free WiFi, laundry facilities, and a private deck with a BBQ, providing easy access to a garden and lush black bamboo surroundings. The apartment's prime location makes it accessible to downtown Seattle in about fifteen minutes by bus, historic Ballard in ten minutes or a short walk to Whole Foods, Peet's Coffee, and the scenic top of Queen Anne Hill, all within a moderate walking distance.""]"
2,"{""(\""The Space This is a 1000 square foot, two bedroom, one bathroom house.\"",1)"",""(\""Start the morning off right with some coffee on the deck.\"",0.94)""}","[""This document highlights a welcoming, light-filled home featuring a 1000 square foot two-bedroom, one-bathroom layout with a spacious kitchen and two large indoor dining areas. The residence offers quick access to downtown and coffee shops, making it convenient for enjoying Seattle's dining scene. Guests can savor the morning with coffee on a west-facing deck or explore local eateries like Storyville Coffee Company and The Seattle Grind. The home encourages outdoor activities in nearby parks and offers a variety of lunch options in the vicinity, promoting a rich local experience.""]"


**Store Description Summary in the Database**

In [3]:
ALTER TABLE listings
ADD COLUMN summary text;

WITH batch_cte AS (
    SELECT azure_cognitive.summarize_abstractive(ARRAY(SELECT description FROM listings ORDER BY id), 'en', batch_size => 25) AS summary
),
summary_cte AS (
    SELECT
        ROW_NUMBER() OVER () AS id,
        ARRAY_TO_STRING(summary, ',') AS summary
    FROM batch_cte
)
UPDATE listings AS l
SET summary = s.summary
FROM summary_cte AS s
WHERE l.id = s.id;

ALTER TABLE

UPDATE 50

Total execution time: 00:03:00.716

<span style="color: rgb(31, 35, 40); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, &quot;Noto Sans&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px; background-color: rgb(255, 255, 255);"><b>Displaying a summary of all reviews for a property</b></span>

In [5]:
SELECT unnest(azure_cognitive.summarize_abstractive(reviews_combined, 'en')) AS review_summary
FROM (
    -- Combine all reviews for a listing
    SELECT string_agg(comments, ' ') AS reviews_combined
    FROM reviews
    WHERE listing_id = 1
)

(1 row(s) affected)

Total execution time: 00:00:13.666

review_summary
"Guests consistently praise the Queen Anne neighborhood and the host's accommodations in Seattle, highlighting the apartment'ded spaciousness, modern aesthetic, and convenient location near downtown. Visitors commend the comfortable bed, cleanliness, and amenities, including a washer and dryer, with some noting the private patio space as a bonus. However, there are a few negative aspects mentioned, such as potential mustiness in a basement-style space, street noise, and limited towels. Despite these concerns, the overall tone suggests that the stay was enjoyable, with the host being described as welcoming and helpful, making the apartment a great base for exploring Seattle. The accommodations offer a home-away-from-home experience, with easy access to the city's attractions via public transportation. The feedback reflects a positive impression of the stay, with a few minor drawbacks that do not seem to detract significantly from the overall experience."
